In [ ]:
%spark.conf

SPARK_HOME  /usr/local/spark
PYSPARK_PYTHON /usr/bin/python3
spark.pyspark.python  /usr/bin/python3

# set driver memory to 8g
spark.driver.memory 8g

# set executor number to be 3
spark.executor.instances  3

# set executor memory 4g
spark.executor.memory  2g

# Any other spark properties can be set here. Here's avaliable spark configruation you can set. (http://spark.apache.org/docs/latest/configuration.html)

In [1]:
%pyspark
g_sdf = spark.read.format("jdbc")\
    .option("url", "jdbc:mysql://j7d108.p.ssafy.io:3306/gamulgamul_test3?useSSL=false&characterEncoding=UTF-8&useUnicode=true&allowPublicKeyRetrieval=true&serverTimezone=Asia/Seoul") \
    .option("driver", "com.mysql.cj.jdbc.Driver")\
    .option("query", "select goods_id, name from goods") \
    .option("user", "ji")\
    .option("password", "cvgkbhs234r#8tdx").load()
    
g_sdf.show()
g_name = g_sdf.toPandas()['name']
g_name

In [2]:
%pyspark
import pandas as pd
import numpy as np
import datetime
from tqdm import tqdm
import os
import sys
import urllib.request
import json, pprint
import pickle

### api 키
client_id = "a8DDB5LJUVwh8_MWPtSj"
client_secret = "hKlWjoMufh"


def get_goods_cheap_url(goods_name):
  encText = urllib.parse.quote(goods_name)
  url = "https://openapi.naver.com/v1/search/shop.json?exclude=used:rental:cbshop&start=1&display=30&sort=sim&query=" + encText 
  request = urllib.request.Request(url)
  request.add_header("X-Naver-Client-Id",client_id)
  request.add_header("X-Naver-Client-Secret",client_secret)
  response = urllib.request.urlopen(request)
  rescode = response.getcode()

  if(rescode==200):
    response_body = response.read()
    res = json.loads(response_body.decode('utf-8'))
  else:
    print("Error Code:" + rescode)
    return
    
  id = goodsId[goods_name]
  if not res['items']:
    return id, None
  cheap = sorted(res['items'], key= lambda x: int(x['lprice']))[0]
  return id, cheap.get('link')

In [3]:
%pyspark
for name in g_name:
    print(name)

In [4]:
%pyspark
with open('/DB_data/goodsName_goodsId.pickle', 'rb') as f:
    goodsId = pickle.load(f)
goodsId

In [5]:
%MySQL
select * from goods

In [6]:
%pyspark
import requests
url = "http://3.36.106.26:8081/api/notebook/run/2HD9NBXQX/paragraph_1664176216600_9409959"

for name in g_name:
    goods_id, cheap_url = get_goods_cheap_url(name)
    req = requests.post(url, json={"params": {"cheap_url": cheap_url, "goods_id": str(goods_id)}})
    # if req.status != 'OK':
    #     print(req.body.msg)
    

In [7]:
%MySQL
UPDATE goods SET cheap_url = '${cheap_url}' WHERE goods_id = CAST('${goods_id}' AS UNSIGNED);

In [8]:
%MySQL
select * from goods

In [9]:
%pyspark
strange_goods_id = [26, 165, 192, 200, 231, 252, 253, 277, 278, 279, 280, 281, 282, 292, 293, 294, 315]

url = "http://3.36.106.26:8081/api/notebook/run/2HD9NBXQX/paragraph_1664176216600_9409959"

for goods_id in strange_goods_id:
    req = requests.post(url, json={"params": {"cheap_url": None, "goods_id": str(goods_id)}})
    

In [10]:
%MySQL
-- 검색 결과 없는 것들 이미지 바꾸기
UPDATE goods SET img="https://j7d108.p.ssafy.io/resource/logo.png" WHERE goods_id IN (26, 165, 192, 200, 231, 252, 253, 277, 278, 279, 280, 281, 282, 292, 293, 294, 315)

# 자동화 실행

In [12]:
%pyspark
import pandas as pd
import numpy as np
import datetime
from tqdm import tqdm
import os
import sys
import urllib.request
import json, pprint
import pickle

# goodsName_goodsId pickle load
with open('/DB_data/goodsName_goodsId.pickle', 'rb') as f:
    goodsId = pickle.load(f)

# select goods_id, name from goods
g_sdf = spark.read.format("jdbc")\
    .option("url", "jdbc:mysql://j7d108.p.ssafy.io:3306/gamulgamul_test3?useSSL=false&characterEncoding=UTF-8&useUnicode=true&allowPublicKeyRetrieval=true&serverTimezone=Asia/Seoul") \
    .option("driver", "com.mysql.cj.jdbc.Driver")\
    .option("query", "select goods_id, name from goods") \
    .option("user", "ji")\
    .option("password", "cvgkbhs234r#8tdx").load()
    
# name to list
g_name = g_sdf.toPandas()['name']


### api 키
client_id = "a8DDB5LJUVwh8_MWPtSj"
client_secret = "hKlWjoMufh"

# get Naver API cheap_url by goods_name function
def get_goods_cheap_url(goods_name):
  encText = urllib.parse.quote(goods_name)
  url = "https://openapi.naver.com/v1/search/shop.json?exclude=used:rental:cbshop&start=1&display=30&sort=sim&query=" + encText 
  request = urllib.request.Request(url)
  request.add_header("X-Naver-Client-Id",client_id)
  request.add_header("X-Naver-Client-Secret",client_secret)
  response = urllib.request.urlopen(request)
  rescode = response.getcode()

  if(rescode==200):
    response_body = response.read()
    res = json.loads(response_body.decode('utf-8'))
  else:
    print("Error Code:" + rescode)
    return
    
  id = goodsId[goods_name]
  if not res['items']:
    return id, None
  cheap = sorted(res['items'], key= lambda x: int(x['lprice']))[0]
  return id, cheap.get('link')

# automatical run paragraph to update sql goods table cheap_url column
import requests
url = "http://3.36.106.26:8081/api/notebook/run/2HD9NBXQX/paragraph_1664176216600_9409959"

for name in g_name:
    goods_id, cheap_url = get_goods_cheap_url(name)
    req = requests.post(url, json={"params": {"cheap_url": cheap_url, "goods_id": str(goods_id)}})

# missing values update sql goods table cheap_url column as None 
strange_goods_id = [26, 165, 192, 200, 231, 252, 253, 277, 278, 279, 280, 281, 282, 292, 293, 294, 315]
for goods_id in strange_goods_id:
    req = requests.post(url, json={"params": {"cheap_url": None, "goods_id": str(goods_id)}})

In [13]:
%pyspark
requests.post("http://3.36.106.26:8081/api/notebook/run/2HD9NBXQX/paragraph_1664179874437_1108003918")

In [14]:
%MySQL
UPDATE goods SET product_id=37 WHERE goods_id=94